<a href="https://colab.research.google.com/github/HenriARM/ML/blob/master/sentiment-analysis/sentiment-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis of clothes reviews using Word2Vec and LSTM

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Data fetching

import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/sentiment-analysis/reviews.csv', delimiter=',', encoding='latin-1').dropna() # drop empty ones 
reviews = np.array(data["Review Text"])
recommendations = np.array(data["Recommended IND"])
ratings = np.array(data["Rating"])

In [0]:
# Data Processing
from string import punctuation

# convert to lower case and remove punctuation
f_clean = np.vectorize(lambda t: ''.join([c for c in t.lower() if c not in punctuation]))
reviews = f_clean(reviews)
print(reviews)

['i had such high hopes for this dress and really wanted it to work for me i initially ordered the petite small my usual size but i found this to be outrageously small so small in fact that i could not zip it up i reordered it in petite medium which was just ok overall the top half was comfortable and fit nicely but the bottom half had a very tight under layer and several somewhat cheap net over layers imo a major design flaw was the net over layer sewn directly into the zipper  it c'
 'i love love love this jumpsuit its fun flirty and fabulous every time i wear it i get nothing but great compliments'
 'this shirt is very flattering to all due to the adjustable front tie it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan love this shirt'
 ...
 'got this in the petite xs in mint the color is gorgeous its a really bright mint that complements my super fair skin really well though it would work for pretty much any skin tone its still a g

In [0]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
review_lines = list()

word_sentences = list()
for review in reviews:
  review = review.split()
  word_sentences.append([word for word in review if word.isalpha()])

In [0]:
import gensim

model = gensim.models.Word2Vec(sentences=word_sentences, window=5, workers=4, min_count=1, size=64)
words = list(model.wv.vocab)

In [0]:
# Test Word2Vec
model.wv.most_similar("terrible")
model.wv.most_similar(positive=['orange', 'water'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('heather', 0.8220078945159912),
 ('rust', 0.8116054534912109),
 ('burgundy', 0.8107340335845947),
 ('beige', 0.8016921877861023),
 ('poppy', 0.7991843223571777),
 ('mauve', 0.79749596118927),
 ('background', 0.7955273389816284),
 ('peach', 0.7941948771476746),
 ('faded', 0.7933332920074463),
 ('royal', 0.7912448048591614)]

In [0]:
# TODO: add save word2wec dictionart
# model["orange"].size

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


64

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(word_sentences)
sequences = tokenizer_obj.texts_to_sequences(word_sentences)

word_index = tokenizer_obj.word_index
print(len(word_index))
review_pad = pad_sequences(sequences, maxlen=300)
sentiment = recommendations
print(sentiment.shape)

17103
(19252,)


In [0]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 64))

for word, i in word_index.items():
  if i > num_words:
    continue
  embedding_vector = word_index.get(word)
  if embedding_vector is not None:
    #words not found = 0000000
    embedding_matrix[i] = embedding_vector



In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

#model here
#TODO change
model = Sequential()
embedding_layer = Embedding(
    num_words,
    64,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=max_length,
    trainable=False
)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import utils
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(review_pad, recommendations, test_size=0.2)

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
model.fit(x_train, y_train, epochs=1, batch_size=50, verbose=1)

Epoch 1/1
15401/15401 [==============================] - 66s 4ms/step - loss: 0.4721 - acc: 0.8182
